In [3]:
import sys
import os
from os import sep
from os.path import dirname, realpath, splitext
from pathlib import Path
from functools import partial
from itertools import product
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which sis why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('inspect_raw.ipynb', 'raw' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask import delayed, compute
from numba import jit, vectorize, float64, uint
from sortedcontainers import SortedList, SortedSet 

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option('display.max_rows', 500)
# pd.set_option('display.height', 500)
pd.set_option('display.max_columns', 50)

from common_util import DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, DT_BIZ_DAILY_FREQ, list_get_dict, inner_join, right_join, get_custom_biz_freq_df, query_df, search_df, chained_filter, recursive_dict, benchmark
from common_util import RECON_DIR, load_df, load_json, abs_df, df_count, df_value_count, outer_join, left_join, getcon, string_df_join_to_ser, is_df, is_ser, count_nn_nz_df, pairwise, cust_count, get_pardir_from_path
from raw.common import default_pathsfile
from data.data_api import DataAPI
from data.access_util import df_getters as dg, col_subsetters2 as cs2

## Load Raw

In [18]:
accessors = ['raw', 'raw']
au_dg, au_cs = list_get_dict(dg, accessors), list_get_dict(cs2, accessors)
paths, recs, dfs = DataAPI.lazy_load(au_dg, au_cs)

accessors = ['raw', 'id_rm']
au_dg, au_cs = list_get_dict(dg, accessors), list_get_dict(cs2, accessors)
rm_paths, rm_recs, rm_dfs = DataAPI.lazy_load(au_dg, au_cs)

In [14]:
mkt_v3_qd = {
    'exact': [],
    'startswith': ['mkt_v3'],
    'endswith': [],
    'regex': [],
    'exclude': None
}
etf_v3_qd = {
    'exact': [],
    'startswith': ['etf_v3'],
    'endswith': [],
    'regex': [],
    'exclude': None
}
qds = {'mkt_v3': mkt_v3_qd, 'etf_v3': etf_v3_qd}

In [17]:
for path in filter(lambda p: p[-1]!='raw_trmi', paths):
    display(path)
    raw_df = list_get_dict(dfs, path).compute()
    if (path[-1] == 'raw_trmi_v3'):
        continue
        for v3_name, v3_qd in qds.items():
            v3_cols = chained_filter(raw_df.columns, v3_qd)
            display(v3_name, len(v3_cols), v3_cols)
    else:
        display(len(raw_df.columns), raw_df.columns)
        display(raw_df.describe())

['dow_jones', 'raw', 'raw_pba']

5

Index(['pba_open', 'pba_high', 'pba_low', 'pba_close', 'pba_avgPrice'], dtype='object')

,pba_open,pba_high,pba_low,pba_close,pba_avgPrice
count,39863.000000,39863.000000,39863.000000,39863.000000,39863.000000
mean,12456.870574,12479.931850,12433.575578,12457.318090,12457.143081
std,3596.053185,3594.847536,3597.365748,3596.287092,3596.216907
min,6482.620000,6547.130000,6470.110000,6482.300000,6517.557300
25%,10149.905000,10173.860000,10126.615000,10149.945000,10150.470850
50%,11080.800000,11106.380000,11057.160000,11080.910000,11080.389800
75%,13955.010000,13974.475000,13940.015000,13956.415000,13957.476100
max,25308.400000,25311.990000,25289.860000,25299.160000,25298.002500


['sp_500', 'raw', 'raw_pba']

...

,etf_v2_buzz_N,etf_v2_relativeBuzz_N,etf_v2_sentiment_N,etf_v2_optimism_N,etf_v2_fear_N,etf_v2_joy_N,etf_v2_trust_N,etf_v2_violence_N,etf_v2_conflict_N,etf_v2_gloom_N,etf_v2_stress_N,etf_v2_timeUrgency_N,etf_v2_uncertainty_N,etf_v2_emotionVsFact_N,etf_v2_longShort_N,etf_v2_longShortForecast_N,etf_v2_priceDirection_N,etf_v2_priceForecast_N,etf_v2_volatility_N,etf_v2_loveHate_N,etf_v2_anger_N,etf_v2_debtDefault_N,etf_v2_innovation_N,etf_v2_marketRisk_N,etf_v2_analystRating_N,...,etf_v2_conflict,etf_v2_gloom,etf_v2_stress,etf_v2_timeUrgency,etf_v2_uncertainty,etf_v2_emotionVsFact,etf_v2_longShort,etf_v2_longShortForecast,etf_v2_priceDirection,etf_v2_priceForecast,etf_v2_volatility,etf_v2_loveHate,etf_v2_anger,etf_v2_debtDefault,etf_v2_innovation,etf_v2_marketRisk,etf_v2_analystRating,etf_v2_dividends,etf_v2_earningsForecast,etf_v2_fundamentalStrength,etf_v2_layoffs,etf_v2_litigation,etf_v2_managementChange,etf_v2_managementTrust,etf_v2_mergers
count,175409.000000,175409.000000,175409.000000,175037.000000,170465.000000,166653.000000,174575.000000,175188.000000,175406.000000,174995.000000,175355.000000,175293.000000,174191.000000,175409.000000,174738.000000,155809.000000,171584.000000,126983.000000,175264.000000,170634.000000,168728.000000,169153.000000,165836.000000,175391.000000,146790.000000,...,175409.000000,175409.000000,175409.000000,175409.000000,175409.000000,175409.000000,175409.000000,175406.000000,175409.000000,175388.000000,175408.000000,175408.000000,175408.000000,175389.000000,175334.000000,175409.000000,175303.000000,172388.000000,175399.000000,175408.000000,174269.000000,175407.000000,175292.000000,175403.000000,175404.000000
mean,19822.237026,0.048756,-0.089851,-0.004709,0.005492,0.005063,0.000279,0.040871,0.016900,0.033347,0.061330,0.012779,0.014540,0.144877,0.002116,0.000102,0.003918,0.000278,0.029947,0.004054,0.004447,0.008193,0.018318,-0.003300,0.000349,...,0.007865,0.035397,0.060930,0.013328,0.023947,0.157566,0.000023,0.000061,0.003767,0.000550,0.027029,0.005737,0.010195,0.004457,0.003218,-0.004410,0.000701,0.000322,0.001459,0.006104,0.000598,0.005597,0.001018,-0.000684,0.004283
std,20852.815966,0.018263,0.085223,0.021370,0.003242,0.003466,0.004519,0.030893,0.013943,0.013470,0.014051,0.010283,0.005623,0.020459,0.008551,0.001783,0.009524,0.001734,0.009133,0.003913,0.003235,0.008522,0.035190,0.011756,0.001798,...,0.006312,0.005941,0.006569,0.006510,0.004448,0.011245,0.005057,0.000644,0.003331,0.000601,0.008437,0.002054,0.002816,0.002746,0.002383,0.005816,0.001363,0.001044,0.002323,0.004398,0.000573,0.002769,0.000515,0.001336,0.002411
min,15.000000,0.004057,-0.433735,-0.178849,-0.007092,-0.007294,-0.070161,-0.023256,-0.100000,-0.008333,-0.005015,-0.086433,-0.018182,-0.021555,-0.210526,-0.059737,-0.155642,-0.025205,-0.006109,-0.022272,-0.013937,-0.033309,-0.008778,-0.066361,-0.033632,...,-0.051814,0.004805,0.008226,-0.010909,0.003498,0.030288,-0.084897,-0.008111,-0.013650,-0.006974,0.008189,-0.004698,0.001162,-0.000318,0.000063,-0.025516,-0.017849,-0.005211,-0.020199,-0.064215,-0.000371,0.000000,-0.003891,-0.050758,0.000101
25%,2513.300000,0.036234,-0.152660,-0.013225,0.003758,0.002978,-0.001765,0.021208,0.007640,0.024262,0.053086,0.006685,0.010975,0.132946,-0.001568,-0.000304,-0.000607,-0.000120,0.025120,0.002190,0.002465,0.003662,0.003435,-0.010526,-0.000259,...,0.003788,0.031908,0.057025,0.010026,0.021231,0.151576,-0.002822,-0.000263,0.001845,0.000258,0.023505,0.004647,0.008238,0.002962,0.001439,-0.008622,0.000147,0.000027,0.000466,0.003524,0.000268,0.003935,0.000689,-0.001333,0.002444
50%,11556.700000,0.047055,-0.094408,-0.003536,0.004882,0.004699,0.000790,0.030530,0.014586,0.031000,0.060718,0.013249,0.014422,0.143896,0.002026,0.000131,0.002961,0.000197,0.029786,0.003488,0.003658,0.005779,0.006021,-0.003930,0.000380,...,0.007267,0.035551,0.060940,0.012056,0.023828,0.159520,-0.000358,0.000057,0.003337,0.000478,0.025502,0.005489,0.010461,0.003982,0.002511,-0.005528,0.000528,0.000140,0.001056,0.005144,0.000433,0.

['dow_jones', 'raw', 'raw_trmi_v3']

['sp_500', 'raw', 'raw_trmi_v3']

['nasdaq_100', 'raw', 'raw_trmi_v3']

['russell_2000', 'raw', 'raw_trmi_v3']